In [1]:
import os
import pandas as pd
import numpy as np
import re, operator
from operator import itemgetter
from itertools import islice

# stats

from scipy import stats
from scipy import special
from scipy.spatial.distance import cosine
from scipy.stats import zipf

# dataviz
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots




In [2]:
def read_fcu(filename, corpus_name):
    
    '''
    INPUT :
    OUTPUT :
    '''
    
    # description du corpus
    print("========> Counting bsv..")
    total = 0
    for root, dirs, files in os.walk(corpus_name):
        total += len(files)
    print("\nTotal : {} bsv dans le corpus".format(total))

    
    # create df
    print("\n========> Creating a dataframe..")
    df = pd.read_csv(filename, delimiter="\t", encoding='utf-8')

    
    # short descriptions
    print("\n========> Making short descriptions..")
    print("\nPrefLabel :\n")
    print(df['prefLabel'].describe())
    print("\n\nBSV :\n")
    print(df['bsv'].describe())
    #print("\n\nLength :\n")
    #print(df['length'].describe())
    
    # save dictionary of extracted labels
    label_counter = df['prefLabel'].value_counts()
    label_dict = label_counter.to_dict()
    df_freq = pd.DataFrame.from_dict(label_dict,orient='index', columns=['count'])
    #df_freq.reset_index(level='prefLabel')
    df_freq = df_freq.reset_index().rename({'index':'prefLabel'}, axis = 'columns')
    #df_freq['prefLabel'] = df_freq.index
    print("\nTop 10 labels :\n{}".format(df_freq[:10]))

    


    return df, df_freq



def show_frequencies(df_freq):
    
    '''
    INPUT :
    OUTPUT :
    '''
    
    # show scatter graph
    fig = px.scatter(df_freq, x=df_freq['prefLabel'], y=df_freq['count'])
    fig.show()
    
    # show tree graph
    fig = px.treemap(df_freq, path=df_freq , values='count')
    fig.show()
    
    return True




def show_differences(df1, df2):
    
    '''
    INPUT :
    OUTPUT :
    '''
    
    print(df1['prefLabel'].isin(df2['prefLabel']).value_counts())
    print(df1[~df1['prefLabel'].isin(df2['prefLabel'])])
    
    return True



def scatter_matrix(df, list_of_dimensions):
    
    '''
    Scatter plots show how much one variable is affected by another or the relationship between them with the help of dots in two dimensions. 
    Scatter plots are very much like line graphs in the concept that they use horizontal and vertical axes to plot data points
    
    INPUT :
    OUTPUT :
    '''
    
    fig = px.scatter_matrix(df,
    dimensions=list_of_dimensions,
    color=df.index,
    title="Scatter matrix",
    labels={col:col.replace('_', ' ') for col in df.columns}) # remove underscore
    fig.update_traces(diagonal_visible=True)
    fig.show()
    return True



def cosine_similarity(col1, col2):
    '''
    The cosine measure similarity is a similarity metric that depends on envisioning user preferences as points in space.  
    The cosine of a small angle is near 1, and the cosine of a large angle near 180 degrees is close to –1. 
    This is good, because small angles should map to high similarity, near 1, and large angles should map to near –1
    
    INPUT :
    OUTPUT :
    '''
    
    result = (1 - cosine(col1, col2))
    print(result)
    return result


def load_features(filepath):
    '''
    INPUT :
    OUTPUT :
    '''
    # read csv file
    df = pd.read_csv(filepath, sep = '\t', encoding = 'utf8', names = ['bsv', 'prefLabel', 'score'])
    grouped = df.groupby("bsv")
    for name,group in grouped:
        print(name)
        print(group)
    
    
    # transpose df to a tf-idf matrix
    features = df.pivot(index='prefLabel', columns='bsv', values='score') 
    # replace NaN to null
    features = features.fillna(0)
    
    return features

In [25]:
fcu, fcu_freq  = read_fcu("../output/d2kab/fcu-baseline.csv", "../resources/Corpus-v-12-04-21/train/")

========> Counting bsv..

Total : 230 bsv dans le corpus

========> Creating a dataframe..

========> Making short descriptions..

PrefLabel :

count      7861
unique      162
top       vigne
freq       1051
Name: prefLabel, dtype: object


BSV :

count                                                  7861
unique                                                  230
top       BSV_legumes_allium_pomme_de_terre_AURA_2019-14...
freq                                                    191
Name: bsv, dtype: object

Top 10 labels :
           prefLabel  count
0              vigne   1051
1              colza    654
2               chou    369
3               maïs    360
4             tomate    346
5    arbuste à baies    286
6            carotte    254
7  culture légumière    237
8         maraîchage    219
9            poireau    204


In [87]:
#header_true = fcu.loc[fcu['html tag'].str.startswith('H', na=False)]
fcu['header'] = 'false'
fcu.loc[fcu['html tag'].str.startswith('H', na=False),'header'] = 'true'
header_true = fcu[['bsv', 'number of words','prefLabel', "header"]]
header_true

,bsv,number of words,prefLabel,header
0,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false
1,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,maïs,false
2,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false
3,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false
4,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false
...,...,...,...,...
7856,bsv_viti_mp_gaillac_n18_30072019_cle0fe8aa,4306,gel,false
7857,bsv_viti_mp_gaillac_n18_30072019_cle0fe8aa,4306,gel,false
7858,bsv_viti_mp_gaillac_n18_30072019_cle0fe8aa,4306,gel,false
7859,bsv_viti_mp_gaillac_n18_30072019_cle0fe8aa,4306,gel,false


In [104]:
scores = pd.read_csv("../output/d2kab/bm25-baseline.csv", sep = '\t', encoding = 'utf8', names = ['bsv', 'prefLabel', 'score'])
result = pd.merge(header_true, scores, on=["bsv", "prefLabel"])
result

,bsv,number of words,prefLabel,header,score
0,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false,1.760917
1,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false,1.760917
2,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false,1.760917
3,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false,1.760917
4,20180911_bsv_grandes_cultures-26_cle0f15a8,2432,colza,false,1.760917
...,...,...,...,...,...
7856,bsv_viti_mp_gaillac_n18_30072019_cle0fe8aa,4306,céréale à paille,false,2.865633
7857,bsv_viti_mp_gaillac_n18_30072019_cle0fe8aa,4306,gel,false,7.465499
7858,bsv_viti_mp_gaillac_n18_30072019_cle0fe8aa,4306,gel,false,7.465499
7859,bsv_viti_mp_gaillac_n18_30072019_cle0fe8aa,4306,gel,false,7.465499


In [108]:
#grouped = result.groupby(['bsv','prefLabel', 'score','header']).size()

test = pd.DataFrame({'count' : result.groupby( ['bsv','prefLabel', 'score','header'] ).size()}).reset_index()
#pd.DataFrame({'header' : result.groupby( ['bsv','prefLabel', 'score'] )['header'].agg('sum')}).reset_index()
#grouped
# Use transform to sum the `size` by the first column only.
test['total'] = test.groupby(['bsv', 'prefLabel','score'])['count'].transform('sum')
test['percentage'] = test.groupby(['bsv', 'prefLabel','score'])['count'].transform(lambda x: x/x.sum() * 100)

#test['perc'] = test.count / test.total 
test

,bsv,prefLabel,score,header,count,total,percentage
0,20180802_bsvmaraichage_19_cle04c2cf,asperge,3.886483,false,2,2,100.000000
1,20180802_bsvmaraichage_19_cle04c2cf,aubergine,3.543583,false,1,3,33.333333
2,20180802_bsvmaraichage_19_cle04c2cf,aubergine,3.543583,true,2,3,66.666667
3,20180802_bsvmaraichage_19_cle04c2cf,carotte,3.288598,false,4,7,57.142857
4,20180802_bsvmaraichage_19_cle04c2cf,carotte,3.288598,true,3,7,42.857143
...,...,...,...,...,...,...,...
2597,char_gdes_Cultures_no26_du_14-08-19_cle0fb929,pomme de terre,3.761328,false,2,2,100.000000
2598,char_gdes_Cultures_no26_du_14-08-19_cle0fb929,pomme de terre potagère,6.427251,false,1,1,100.000000
2599,ge_houblon_no6_du_19-06-19_cle41558d,floriculture,1.904196,false,1,1,100.000000
2600,ge_houblon_no6_du_19-06-19_cle41558d,houblon,10.288002,false,3,4,75.000000
